In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment', 'installment_rate', 'personal_status', 'debtors', 'residencesince', 'property', 'age', 'install_plans', 'housing', 'existing_credits', 'job', 'maintenance_paying_people', 'telephone', 'foreign_worker', 'result']



In [3]:
df = pd.read_table('german.data', names=cols, sep=" ", index_col=False)

In [4]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,property,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
def preprocess_german(df): 
    df['status'] = df['status'].map({'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}).astype(int)
    df['credit_hist'] = df['credit_hist'].map({'A34': 0, 'A33': 1, 'A32': 2, 'A31': 3, 'A30': 4}).astype(int)
    df.loc[(df['credit_amt'] <= 2000), 'credit_amt'] = 0
    df.loc[(df['credit_amt'] > 2000) & (df['credit_amt'] <= 5000), 'credit_amt'] = 1
    df.loc[(df['credit_amt'] > 5000), 'credit_amt'] = 2   
    df.loc[(df['duration'] <= 12), 'duration'] = 0
    df.loc[(df['duration'] > 12) & (df['duration'] <= 24), 'duration'] = 1
    df.loc[(df['duration'] > 24) & (df['duration'] <= 36), 'duration'] = 2
    df.loc[(df['duration'] > 36), 'duration'] = 3
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['savings'] = df['savings'].map({'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}).astype(int)
    df['employment'] = df['employment'].map({'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}).astype(int)    
    df['gender'] = df['personal_status'].map({'A91': 1, 'A92': 0, 'A93': 1, 'A94': 1, 'A95': 0}).astype(int)
    df['debtors'] = df['debtors'].map({'A101': 0, 'A102': 1, 'A103': 2}).astype(int)
    df['property'] = df['property'].map({'A121': 3, 'A122': 2, 'A123': 1, 'A124': 0}).astype(int)        
    df['install_plans'] = df['install_plans'].map({'A141': 1, 'A142': 1, 'A143': 0}).astype(int)
    df['job'] = df['job'].map({'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}).astype(int)    
    df['telephone'] = df['telephone'].map({'A191': 0, 'A192': 1}).astype(int)
    df['foreign_worker'] = df['foreign_worker'].map({'A201': 1, 'A202': 0}).astype(int)
    pd.get_dummies(df, columns=['purpose', 'housing'], drop_first=True)
    
    return df



In [6]:
X = preprocess_german(df);

In [7]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,A43,0,4,4,4,A93,0,...,1,0,A152,2,2,1,1,1,1,1
1,1,3,2,A43,2,0,2,2,A92,0,...,0,0,A152,1,2,1,0,1,2,0
2,3,0,0,A46,1,0,3,2,A93,0,...,1,0,A152,1,1,2,0,1,1,1
3,0,3,2,A42,2,0,3,2,A93,2,...,1,0,A153,1,2,2,0,1,1,1
4,0,1,1,A40,1,0,2,3,A93,0,...,1,0,A153,2,2,2,0,1,2,1


In [8]:
df = df.drop(["purpose", "housing"], axis=1)

In [9]:
df = df.drop(["personal_status"], axis=1)

In [10]:
df.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'result',
       'gender'],
      dtype='object')

In [11]:
X = df[['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker',
       'gender']]

In [12]:
y = df[["result"]]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=15)

In [14]:
x_train.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'gender'],
      dtype='object')

In [15]:
df.head()


,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1,1
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,2,0
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,2,1


In [16]:
svm_model = svm.SVC(kernel='linear', C=1.0, probability=True)

In [17]:
y.head()

,result
0,1
1,2
2,1
3,1
4,2


In [18]:
x_train

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,gender
366,3,1,0,1,0,3,3,0,1,1,0,0,2,2,1,0,1,1
302,2,1,0,0,4,3,4,0,2,3,0,1,2,1,2,0,1,1
305,3,0,2,0,3,2,4,0,2,3,0,0,1,2,1,0,1,1
684,1,2,1,2,1,3,1,0,3,2,0,0,2,1,2,1,1,1
348,3,0,1,0,1,2,1,0,2,3,0,0,2,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,1,3,2,2,0,2,2,0,2,1,0,0,1,2,1,1,1,1
645,3,2,1,2,4,1,4,0,4,1,0,0,2,2,1,1,1,1
908,3,1,1,1,0,1,1,0,2,2,1,0,2,1,1,0,1,0
757,2,1,0,0,4,2,3,0,4,0,0,0,2,2,1,1,1,1


In [19]:
svm_model.fit(x_train, y_train.values.ravel())


SVC(kernel='linear', probability=True)

In [20]:
y_pred = svm_model.predict(x_test)

In [21]:
accuracy_score(y_test, y_pred)

0.7566666666666667

In [22]:
x_test.iloc[0]

status                       0
duration                     1
credit_hist                  0
credit_amt                   0
savings                      0
employment                   4
installment_rate             4
debtors                      0
residencesince               3
property                     1
age                          0
install_plans                0
existing_credits             2
job                          2
maintenance_paying_people    1
telephone                    1
foreign_worker               1
gender                       1
Name: 825, dtype: int64

In [23]:
groups_age =  df.groupby(df['age'])

In [24]:
protected_group_age_df = groups_age.get_group(1)

In [25]:
protected_group_age_df.iloc[1]
# According to the result it shows that the division is done correctly as age is coming as 1 for protected group

status                       3
duration                     0
credit_hist                  0
credit_amt                   1
savings                      0
employment                   3
installment_rate             2
debtors                      0
residencesince               3
property                     3
age                          1
install_plans                0
existing_credits             1
job                          1
maintenance_paying_people    2
telephone                    0
foreign_worker               1
result                       1
gender                       1
Name: 2, dtype: int64

In [26]:
non_protected_group_age_df = groups_age.get_group(0)

In [27]:
non_protected_group_age_df.iloc[1]
# According to the result it shows that the division is done correctly as age is coming as 0 for non-protected group

status                       3
duration                     2
credit_hist                  2
credit_amt                   2
savings                      4
employment                   2
installment_rate             2
debtors                      0
residencesince               4
property                     0
age                          0
install_plans                0
existing_credits             1
job                          1
maintenance_paying_people    2
telephone                    1
foreign_worker               1
result                       1
gender                       1
Name: 5, dtype: int64

In [28]:
y_pred = svm_model.predict(x_test)

In [29]:
y_pred

array([1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], d

In [30]:
# There are various fairness metrics defined in the 'Fairness Definitions Explained' paper based on the predicted probabilities and actual outcome

# 1. Test fairness -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f), which can be used for age(protected and non-protected classes)
# 2. Well-calibration -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f) = s, which can be used for age(protected and non-protected classes)
# 3. Balance for positive class -> E(S|Y=1,G=m) = E(S|Y=1,G=f), which can be used for age(protected and non-protected classes)
# 4. Balance for negative class -> E(S|Y=0,G=m) = E(S|Y=0,G=f), which can be used for age(protected and non-protected classes)

In [31]:
# Now, we need to find the P(ŷ = 1 | y = 1, G = 0) and P(ŷ = 1 | y = 1, G = 1) as G is the sensitive attribute here

In [32]:
# Find the average y_pred_proba where the actual outcome Y = 1 for the divided groups 

In [33]:
non_protected_group_age_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,2,0
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1,1
9,1,2,0,2,0,0,4,0,2,1,0,0,2,3,1,0,1,2,1
10,1,0,2,0,0,1,3,0,1,1,0,0,1,2,1,0,1,2,0


In [34]:
non_protected_group_age_credresult_1_df = non_protected_group_age_df.groupby(non_protected_group_age_df['result']).get_group(1)

In [35]:
non_protected_group_age_credresult_1_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1,1
12,1,0,2,0,0,2,1,0,1,1,0,0,1,2,1,1,1,1,0
14,0,1,2,0,0,2,2,0,4,1,0,0,1,2,1,0,1,1,0
17,0,2,4,2,4,1,2,0,3,1,0,1,3,2,1,0,1,1,1


In [36]:
#Drop the result column as the logistic regression model will accept 18 columns as input.
non_protected_group_age_credresult_1_df = non_protected_group_age_credresult_1_df.drop(["result"], axis=1)

In [37]:
# Need to find the predictive probability for the dataframe and then finally find the average
y_pred_non_protected_group_age_credresult_1 = svm_model.predict(non_protected_group_age_credresult_1_df)

In [38]:
non_protected_group_age_credresult_1_df

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,gender
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1
12,1,0,2,0,0,2,1,0,1,1,0,0,1,2,1,1,1,0
14,0,1,2,0,0,2,2,0,4,1,0,0,1,2,1,0,1,0
17,0,2,4,2,4,1,2,0,3,1,0,1,3,2,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,0,2,2,1,0,0,4,0,3,2,0,0,1,3,1,1,1,1
995,3,0,2,0,0,3,3,0,4,3,0,0,1,1,1,0,1,0
996,0,2,2,1,0,2,4,0,4,2,0,0,1,3,1,1,1,1
997,3,0,2,0,0,4,4,0,4,1,0,0,1,2,1,0,1,1


In [39]:
y_pred_non_protected_group_age_credresult_1

array([1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [40]:
#avergae predicted probability came out to be 0.879781 for non protected group

In [41]:
protected_group_age_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1,1
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,2,1
6,3,1,2,1,2,4,3,0,4,2,1,0,1,2,1,0,1,1,1


In [42]:
protected_group_age_credresult_1_df = protected_group_age_df.groupby(protected_group_age_df['result']).get_group(1)

In [43]:
#Drop the result column as the logistic regression model will accept 18 columns as input.
protected_group_age_credresult_1_df = protected_group_age_credresult_1_df.drop(["result"], axis=1)

In [44]:
# Need to find the predict probability for the dataframe and then finally find the average
y_pred_protected_group_age_credresult_1 = svm_model.predict(protected_group_age_credresult_1_df)

In [45]:
y_pred_protected_group_age_credresult_1

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [46]:
#avergae predicted probability came out to be 0.92053 for protected group

In [47]:
# Need to find the predict probability for the dataframe and then finally find the average
y_pred_protected_group_age_credresult_1 = svm_model.predict_proba(protected_group_age_credresult_1_df)

In [48]:
y_pred_protected_group_age_credresult_1

array([[0.87931222, 0.12068778],
       [0.94002105, 0.05997895],
       [0.47084527, 0.52915473],
       [0.8845599 , 0.1154401 ],
       [0.94957565, 0.05042435],
       [0.93054392, 0.06945608],
       [0.85019316, 0.14980684],
       [0.89229701, 0.10770299],
       [0.96034197, 0.03965803],
       [0.9281136 , 0.0718864 ],
       [0.75131598, 0.24868402],
       [0.97019761, 0.02980239],
       [0.61210356, 0.38789644],
       [0.6227973 , 0.3772027 ],
       [0.82010349, 0.17989651],
       [0.85615222, 0.14384778],
       [0.62678182, 0.37321818],
       [0.68797542, 0.31202458],
       [0.91709422, 0.08290578],
       [0.75838794, 0.24161206],
       [0.86231847, 0.13768153],
       [0.93892003, 0.06107997],
       [0.93191409, 0.06808591],
       [0.85019358, 0.14980642],
       [0.59683753, 0.40316247],
       [0.88535245, 0.11464755],
       [0.91184324, 0.08815676],
       [0.92560361, 0.07439639],
       [0.5       , 0.5       ],
       [0.61459903, 0.38540097],
       [0.

In [49]:
svm_model.classes_

array([1, 2], dtype=int64)

In [50]:
y_pred_protected_group_age_credresult_1.transpose()[0].mean()

0.7933953971047462

In [51]:
y_pred_non_protected_group_age_credresult_1 = svm_model.predict_proba(non_protected_group_age_credresult_1_df)

In [52]:
y_pred_non_protected_group_age_credresult_1.transpose()[0].mean()

0.7499981582881334